In [1]:
#Import libs
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2022-08-04 09:00:06.583941: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Load Cifar-10 data-set
(train_im, train_lab), (test_im, test_lab) = tf.keras.datasets.cifar10.load_data()

In [3]:
#### Normalize the images to pixel values (0, 1)
train_im, test_im = train_im/255.0 , test_im/255.0
#### Check the format of the data 
print ("train_im, train_lab types: ", type(train_im), type(train_lab))
#### check the shape of the data
print ("shape of images and labels array: ", train_im.shape, train_lab.shape) 
print ("shape of images and labels array ; test: ", test_im.shape, test_lab.shape)

train_im, train_lab types:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>
shape of images and labels array:  (50000, 32, 32, 3) (50000, 1)
shape of images and labels array ; test:  (10000, 32, 32, 3) (10000, 1)


In [4]:
### One hot encoding for labels 

train_lab_categorical = tf.keras.utils.to_categorical(
    train_lab, num_classes=10, dtype='uint8')

test_lab_categorical = tf.keras.utils.to_categorical(
    test_lab, num_classes=10, dtype='uint8')

In [5]:
#from tensorflow.signal import fft2d
#from tensorflow.math import real, imag

from scipy.fftpack import fft2, ifft2


In [6]:
#New plane for the images
#FFT the image with tensorflow
#Then create 2 new variables for the real and imaginary parts of the image
#Then convert the output arrays to numpy arrays

def convert_to_fft(dataset):
    temp = fft2(dataset)
    real_temp = np.real(temp)
    imag_temp = np.imag(temp)
    return real_temp, imag_temp

In [7]:
train_fft = np.concatenate(convert_to_fft(train_im), axis=3)
test_fft = np.concatenate(convert_to_fft(test_im), axis=3)


In [8]:
# Create the model
model = tf.keras.models.Sequential()

# Add the convolutional layers
# input shape is 32x32x6 (32x32 is the size of the image, 6 is the number of "color" channels)

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 6)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
# Add the fully connected layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.add(tf.keras.layers.Flatten())


2022-08-04 09:00:13.901958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 09:00:13.964128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 09:00:13.964390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 09:00:13.965173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [9]:
# Compile the model
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
                

In [10]:
np.shape(train_lab_categorical)

(50000, 10)

In [11]:
# Train the model
model.fit(train_fft, train_lab_categorical, epochs=20, batch_size=16, validation_data=(test_fft, test_lab_categorical), workers=4)

Epoch 1/20


2022-08-04 09:00:16.896768: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-08-04 09:00:17.855162: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-04 09:00:17.935054: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 19s 5ms/step - loss: 1.3527 - accuracy: 0.5228 - val_loss: 1.1153 - val_accuracy: 0.6114
Epoch 2/20
3125/3125 [==============================] - 16s 5ms/step - loss: 1.0173 - accuracy: 0.6430 - val_loss: 0.9399 - val_accuracy: 0.6715
Epoch 3/20
3125/3125 [==============================] - 16s 5ms/step - loss: 0.9142 - accuracy: 0.6783 - val_loss: 0.9490 - val_accuracy: 0.6704
Epoch 4/20
3125/3125 [==============================] - 16s 5ms/step - loss: 0.8469 - accuracy: 0.7015 - val_loss: 0.8945 - val_accuracy: 0.6846
Epoch 5/20
3125/3125 [==============================] - 16s 5ms/step - loss: 0.7939 - accuracy: 0.7217 - val_loss: 0.9241 - val_accuracy: 0.6829
Epoch 6/20
3125/3125 [==============================] - 17s 5ms/step - loss: 0.7528 - accuracy: 0.7349 - val_loss: 0.9202 - val_accuracy: 0.6871
Epoch 7/20
3125/3125 [==============================] - 17s 5ms/step - loss: 0.7129 - accuracy: 0.7475 - val_loss: 0.8975 - val_accuracy: 0.6

: 